<a href="https://colab.research.google.com/github/skwolvie/assets/blob/main/resume_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP AND INSTALL ENV

In [1]:
!apt install -qq enchant
!apt install mecab
!apt install libmecab-dev
!apt install mecab-ipadic-utf8
!apt install file
!pip install easyocr opencv-python pyenchant joblib tqdm neologdn pyspellchecker mecab-python3

The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 37 not upgraded.
Need to get 1,312 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../0-libtext-iconv-perl_1.7-5build6_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-5build6) ...
Selecting previously unselected package libaspe

In [4]:
# neologd settings　・辞書インストール
!git clone https://github.com/neologd/mecab-ipadic-neologd.git
%cd mecab-ipadic-neologd
!bin/install-mecab-ipadic-neologd -y
%cd ../

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 9210, done.
remote: Total 9210 (delta 0), reused 0 (delta 0), pack-reused 9210
Receiving objects: 100% (9210/9210), 452.82 MiB | 38.04 MiB/s, done.
Resolving deltas: 100% (5319/5319), done.
/content/mecab-ipadic-neologd
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-

In [5]:
%%writefile /usr/local/etc/mecabrc
;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n

Writing /usr/local/etc/mecabrc


# Setup Scripts

In [6]:
import os
os.getcwd()

'/content'

In [7]:
os.chdir(r"/content/drive/My Drive/resumeOCR")
os.getcwd()

'/content/drive/My Drive/resumeOCR'

In [8]:
os.listdir()

['resume_ceo.png',
 'resume1.png',
 'constants.py',
 'constraints.py',
 '__pycache__',
 '.ipynb_checkpoints',
 'spell.py',
 'words.txt',
 'ocr.py',
 'output.csv',
 'output.gsheet']

In [9]:
#constants.py
headers = ['', '不可',
        'メールアドレス'
        '氏名','名前', '苗字',
        '現住所', '住所', '連絡先','居住地',
        'ふりがな','フリガナ', 'りがな',
        '生年月日','誕生日','平成', '昭和','年','月' 
        '性別', 
        'TEL', '電話', '携帯','tel', '連絡先', 'contact', '電話番号','FAX','fax',
        '自己PR',
        '職歴', '就労経験', '現在の就業',
        '学歴','就学状湿','現在の就業・就学状湿',
        '資格', '免許''日本での運転可', '日本での運転可・不可',
        '配偏者'
        ]
        
separators = r'[\\\)）（\\\(\s‐–ー−－-]*?'

ph_regex = r'([（\\\(]?0([1-9]{1}' + separators + r'[1-9]\d{3}|[1-9]{2}' + separators + \
           r'\d{3}|[1-9]{2}\d{1}' + separators + r'\d{2}|[1-9]{2}\d{2}' + separators + \
           r'\d{1})' + separators + r'\d{4}|0[789]0' + separators + r'\d{4}' + separators + \
           r'\d{4}|050' + separators + r'\d{4}' + separators + r'\d{4})'

In [47]:
#constraints.py
from abc import ABC, abstractmethod
from typing import List
import re
from constants import ph_regex
# TODO: we should change constraints to take coordinates as well so we can analyze things like font-size etc.
class Constraint(ABC):
    def filter(self, target):
        raise NotImplementedError("filter is not implemented")

class ContainsAnyString(Constraint):
    def __init__(self, strings_to_test: List[str]):
        super(ContainsAnyString, self).__init__()
        self.strings_to_test = strings_to_test
    def filter(self, target):
        return any([test in target.lower() for test in self.strings_to_test])

class MatchesAnyPattern(Constraint):
    def __init__(self, patterns: List[str]):
        super(MatchesAnyPattern, self).__init__()
        self.patterns = [re.compile(pattern) for pattern in patterns]
    def filter(self, target):
        for pattern in self.patterns:
            if pattern.findall(target):
                return True
        return False

class LambdaTest(Constraint):
    def __init__(self, func):
        super(LambdaTest, self).__init__()
        self.func = func
    def filter(self, target):
        return self.func(target)

header_tests = {
    'untitled': LambdaTest(lambda x: x == ''),
    'name': ContainsAnyString(['氏名', '名前', '苗字']),
    'address': ContainsAnyString(['居住地', '住所', '連絡先']),
    'furigana': ContainsAnyString(['ふりがな', 'フリガナ', 'りがな']),
    'birthday': ContainsAnyString(['生年月日', '誕生日', '平成', '昭和']),
    'gender': ContainsAnyString(['性別']),
    'telephone': ContainsAnyString(['電話', '携帯', 'tel', '連絡先', 'contact', '電話番号']),
    'email': ContainsAnyString(['メールアドレス', 'メール', '連絡先', 'contact', 'email']),
    'work_history': ContainsAnyString(['職歴']),
    'education': ContainsAnyString(['学歴']),
    'qualification': ContainsAnyString(['資格', '免許']),
    'pr': ContainsAnyString(['自己PR']),
    'year': ContainsAnyString(['年']),
    'month': ContainsAnyString(['月']),
    'job': ContainsAnyString(['現在の就業・就学状湿']),
    'driving': ContainsAnyString(['日本での運転可・不可', '日本での運転可'])
}

# Test Name using MeCab Proper proper noun identification
content_tests = {
    'address': MatchesAnyPattern([r'^〒\s*?\d{3}', r'テ\s*?\d{3}', r'[都道府県].*?[市区町村]']),
    'birthday': MatchesAnyPattern([r'\d+(\s+)?年(\s+)?\d+(\s+)?月(\s+)?\d+日生']),
    'gender': MatchesAnyPattern([r'[性別\s]*?[男女]', r'[性別\s]*?女', r'男\s*?.{,1}?\s*?女', r'女\s*?.{,1}?\s*?男', r'^[女男]$']),
    'telephone': MatchesAnyPattern([ph_regex]),
    'email': MatchesAnyPattern([r'\w@\w']),
    'year': MatchesAnyPattern([r"\d\d\d\d"]),
    'month': MatchesAnyPattern([r"\d\d", '\d']),
    'education': ContainsAnyString(['学歴', '香港寝曾大鼻國際鼻院', ]),
    'work':  ContainsAnyString(['職歴']),
     'qualification': ContainsAnyString(['資格', '免許']),            
}

def get_column(text):
    for col, test in header_tests.items():
        if test.filter(text):
            return col
    return None

def content_check(dict):
    keys=dict.keys()
    print(keys)
    for col, test in content_tests.items():
        for key in keys:
            if key==col:
                print(key)
                if test.filter(dict[key]):
                    return dict

    return None

if __name__ == '__main__':
    print(get_column('生年月日'))
    print(content_check({'birthday': 'wtf'}))

birthday
dict_keys(['birthday'])
birthday
None


In [11]:
#spell.py
import MeCab
import neologdn
from typing import List
from collections import defaultdict
from tqdm import tqdm
import joblib
from joblib import Parallel, delayed
import enchant
#enchant.Dict("ja_JA")

tagger = MeCab.Tagger("-F%m\\t --unk-feature 未知語") #"-Owakati"

def tokeit(text):
    text = neologdn.normalize(text, repeat=2)
    text = ''.join([i for i in text if i.isalpha() or i.isspace()])
    result = tagger.parseToNode(text)

    content_words = []
    while result:
        lemma = result.feature.split(",")[6] if len(result.feature.split(",")) > 6 and result.feature.split(",")[6] != "*" else result.surface
        content_words.append((result.surface))
        result = result.next
    return " ".join(content_words)

def editDistance(str1, str2, m, n):
    if m == 0:
        return n
 
    if n == 0:
        return m
 
    if str1[m-1] == str2[n-1]:
        return editDistance(str1, str2, m-1, n-1)
    
    return 1 + min(editDistance(str1, str2, m, n-1),    # Insert
                   editDistance(str1, str2, m-1, n),    # Remove
                   editDistance(str1, str2, m-1, n-1)    # Replace
                   )

def spellcheck(text):
    titles = enchant.PyPWL("words.txt",)
    subs= text.split()
    #subs=tokeit(text).split()
    text_file = open("words.txt", "r")
    headerlist = text_file.read().split('\n')
    for ww, i in tqdm(enumerate(headerlist), total=len(headerlist)):
        for j in subs:
            if 1>=abs(len(i)-len(j))<3:
                if len(j)==2 and editDistance(i, j, len(i), len(j))==1:
                    loc= subs.index(j)
                    subs[loc]=titles.suggest(j, )[0]
                else:
                    if 1>=editDistance(i, j, len(i), len(j))<3:
                        loc= subs.index(j)
                        subs[loc]=titles.suggest(j, )[0]
            else:
                pass
    text=" ".join(subs)
    return text

In [12]:
spellcheck('学 学状湿 現在の就業・学状湿 資格 免許日本での運転 日本での運転可・不 配偏')

100%|██████████| 27/27 [00:08<00:00,  3.24it/s]


'学歴 就学状湿 現在の就業・就学状湿 資格 免許日本での運転可 日本での運転可・不可 配偏'

In [13]:
#ocr.py
import easyocr
import cv2

class OCR:

    def __init__(self, _languages=None):
        languages = ['ja', 'en'] if _languages is None else _languages
        self.reader = easyocr.Reader(languages)

    def read(self, image=None):
        if image is None:
            print('申し訳ありませんが、画像がありません')
            return ''
        else:
            result = self.reader.readtext(image)
            return result

    def img_shape(self):
        return

    @staticmethod
    def print_text(data):
        for index, val in enumerate(data):
            print(f'String detected ({index+1}): {val[0]}')
            print(f'Coordinate detected ({index + 1}): {val[1]}')


if __name__ == "__main__":
    img = cv2.imread('resume1.png')
    img_gaussian = cv2.GaussianBlur(src=img,
                                    ksize=(5, 5),
                                    sigmaX=1,
                                    sigmaY=1)

    ocr = OCR()
    ret_data = ocr.read(img_gaussian)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [14]:
print(ret_data)

[([[215, 270], [408, 270], [408, 348], [215, 348]], '雁歴書', 0.1186076975323952), ([[1073, 286], [1401, 286], [1401, 341], [1073, 341]], '令和 3年8月31日', 0.9943765891565416), ([[1440, 286], [1538, 286], [1538, 341], [1440, 341]], '現在', 0.9945253935731344), ([[239, 385], [423, 385], [423, 437], [239, 437]], 'ふりがな', 0.8596696093327317), ([[847, 381], [1008, 381], [1008, 446], [847, 446]], 'テスト', 0.9971055388450623), ([[237, 459], [330, 459], [330, 516], [237, 516]], '氏名', 0.9992999283477301), ([[721, 545], [882, 545], [882, 613], [721, 613]], 'てすと', 0.9999977975838318), ([[237, 760], [338, 760], [338, 817], [237, 817]], '平成', 0.9999512785907602), ([[363, 763], [447, 763], [447, 815], [363, 815]], '6年', 0.9987959926485194), ([[472, 766], [546, 766], [546, 815], [472, 815]], '4', 0.9774246270100697), ([[585, 763], [727, 763], [727, 815], [585, 815]], '23日生', 0.9982367753982544), ([[763, 760], [944, 760], [944, 817], [763, 817]], '(満27歳)', 0.9940179142602508), ([[1415, 760], [1601, 760], [1601, 

In [48]:
# from ocr import OCR
# from constants import headers
# from spell import spellcheck
import cv2
import sys
import pandas as pd

def is_title(text):
    if text in headers:
        return True
    else:
        return False

def able_to_merge(data_1, data_2, _image_dim=None, margin_x=0.017, margin_y=0.003):
    image_dim = [2483, 3508] if _image_dim is None else _image_dim # testing purposes
    dim_x = image_dim[0]
    dim_y = image_dim[1]
    if (data_2['coordinates'][0][1] in range(data_1['coordinates'][0][1] - int(margin_y * dim_y),
                                             data_1['coordinates'][3][1] + int(margin_y * dim_y)) and
            abs(data_2['coordinates'][0][0] - data_1['coordinates'][1][0]) < int(margin_x * dim_x)):
        return True
    else:
        return False

def merge(data_1, data_2):
    res_data = {
        'text': data_1['text'] + ' ' + data_2['text'],
        'coordinates': [[data_1['coordinates'][0][0], data_1['coordinates'][0][1]],
                        [data_2['coordinates'][1][0], data_2['coordinates'][1][1]],
                        [data_2['coordinates'][2][0], data_1['coordinates'][2][1]],
                        [data_1['coordinates'][3][0], data_1['coordinates'][3][1]]],
        'confidence': (data_1['confidence'] + data_2['confidence']) / 2
    }
    return res_data

def get_merged_data(list_of_data, image_dim=None):
    if image_dim is None:
        image_dim = [2483, 3508]            # testing purposes
    final_list = []
    for i, data_1 in enumerate(list_of_data):
        if data_1['confidence'] == -1:
            continue
        elif is_title(data_1['text']):
            final_list.append(data_1)
        else:
            for j, data_2 in enumerate(list_of_data[i + 1:]):
                if able_to_merge(data_1, data_2, image_dim):
                    data_1 = merge(data_1, data_2)
                    data_2['confidence'] = -1
            final_list.append(data_1)
    return final_list


def get_distance(point_a, point_b):
    distance = pow(((point_a[0] - point_b[0]) ** 2 + (point_a[1] - point_b[1]) ** 2), 1 / 2)
    return distance

class EvaluateResume:
    def __init__(self, _filename='resume1.png'):
        reader = OCR()
        image = cv2.imread(_filename)
        self.image_dim = [image.shape[1], image.shape[0]]
        data_list = reader.read(image)
        list_of_data = [{"wrongtext": '', 'text': '', 'coordinates': [[0, 0], [0, 0], [0, 0], [0, 0]], 'confidence': 0.0}]
        for values in data_list:
            if values[2]>=.90:
                new_data = {"wrongtext": '', 'text': values[1], 'coordinates': values[0], 'confidence': values[2]} # "correcttext": spellcheck(values[1]),
            else:
                new_data = {"wrongtext":values[1] , 'text': spellcheck(values[1]), 'coordinates': values[0], 'confidence': values[2]} # "correcttext": spellcheck(values[1]),
                print(values[1])
                print(new_data)
            list_of_data.append(new_data)
        self.all_data = get_merged_data(list_of_data, self.image_dim)
        pd.DataFrame(list_of_data).to_csv('output.csv', index=False)
    
    def get_image_dimensions(self):
        return self.image_dim

    def get_title(self, data):
        distances = []
        for current_data in self.all_data:
            if not is_title(current_data['text']):
                continue
            else:
                if data['coordinates'][1][0] > current_data['coordinates'][0][0]:
                    distance = get_distance(data['coordinates'][0], current_data['coordinates'][0])
                    base = [data['coordinates'][2][0] + 0.005 * self.image_dim[0],
                            data['coordinates'][2][1] + 0.005 * self.image_dim[1]]
                    if current_data['coordinates'][3][0] < base[0] and current_data['coordinates'][3][1] < base[1]:
                        distances.append([current_data, distance])

        final_distances = sorted(distances, key=lambda l: l[1])
        return final_distances[0][0]

    def get_key_value_pairs(self, data):
        if is_title(data['text']):
            return None
        else:
            val = self.get_title(data)
            res = {get_column(val['text']): data['text']}
            if content_check(res):
                return res

def actual_main():
    all_pairs = []
    evaluator = EvaluateResume(_filename='resume1.png')
    for data in evaluator.all_data:
        res = evaluator.get_key_value_pairs(data)
        all_pairs.append(res)
        pd.DataFrame(all_pairs).to_csv('outputpairs.csv', index=False)
        print(res)

if __name__ == "__main__":
    actual_main()

100%|██████████| 27/27 [00:00<00:00, 8032.79it/s]


雁歴書
{'wrongtext': '雁歴書', 'text': '履歴書', 'coordinates': [[215, 270], [408, 270], [408, 348], [215, 348]], 'confidence': 0.10516757216668074}


100%|██████████| 27/27 [00:00<00:00, 2951.12it/s]


ふりがな
{'wrongtext': 'ふりがな', 'text': 'ふりがな', 'coordinates': [[239, 382], [423, 382], [423, 437], [239, 437]], 'confidence': 0.448096364736557}


100%|██████████| 27/27 [00:00<00:00, 19592.77it/s]


...

株式会社パラダイスプラン (沖縄県宮古島市) 入社
{'wrongtext': '株式会社パラダイスプラン (沖縄県宮古島市) 入社', 'text': '株式会社パラダイスプラン (沖縄県宮古島市) 入社', 'coordinates': [[608, 2670], [1627, 2670], [1627, 2739], [608, 2739]], 'confidence': 0.6709011427846866}


100%|██████████| 27/27 [00:00<00:00, 8469.54it/s]


店_運営
{'wrongtext': '店_運営', 'text': '店_運営', 'coordinates': [[608, 2886], [813, 2886], [813, 2948], [608, 2948]], 'confidence': 0.1612876057624817}


100%|██████████| 27/27 [00:00<00:00, 262.13it/s]


マーケティング 都訳 通訳などを担当
{'wrongtext': 'マーケティング 都訳 通訳などを担当', 'text': 'マーケティング 都訳 通訳などを担当', 'coordinates': [[830, 2885], [1625, 2885], [1625, 2951], [830, 2951]], 'confidence': 0.45542341769175254}
None
dict_keys(['untitled'])
None
dict_keys(['untitled'])
None
dict_keys(['untitled'])
None
None
dict_keys(['furigana'])
None
dict_keys(['furigana'])
None
dict_keys(['furigana'])
None
None
dict_keys(['birthday'])
birthday
{'birthday': '6年 4月 23日生 (満27歳)'}
dict_keys(['birthday'])
birthday
None
None
dict_keys(['furigana'])
None
dict_keys(['furigana'])
None
None
dict_keys(['address'])
address
{'address': 'テ 651-2112'}
None
dict_keys(['address'])
address
{'address': '兵庫県神戸市西区大津和1丁目'}
None
None
dict_keys([None])
None
None
dict_keys(['address'])
address
None
dict_keys(['furigana'])
None
dict_keys(['address'])
address
None
dict_keys([None])
None
None
dict_keys(['year'])
year
None
dict_keys(['year'])
year
None
None
dict_keys(['year'])
year
{'year': '2012'}
dict_keys(['year'])
year
None
dict_keys(['year'])


In [52]:
# from ocr import OCR
# from constants import headers
# from spell import spellcheck
import cv2
import sys
import pandas as pd

def is_title(text):
    if text in headers:
        return True
    else:
        return False

def able_to_merge(data_1, data_2, _image_dim=None, margin_x=0.017, margin_y=0.003):
    image_dim = [2483, 3508] if _image_dim is None else _image_dim # testing purposes
    dim_x = image_dim[0]
    dim_y = image_dim[1]
    if (data_2['coordinates'][0][1] in range(data_1['coordinates'][0][1] - int(margin_y * dim_y),
                                             data_1['coordinates'][3][1] + int(margin_y * dim_y)) and
            abs(data_2['coordinates'][0][0] - data_1['coordinates'][1][0]) < int(margin_x * dim_x)):
        return True
    else:
        return False

def merge(data_1, data_2):
    res_data = {
        'text': data_1['text'] + ' ' + data_2['text'],
        'coordinates': [[data_1['coordinates'][0][0], data_1['coordinates'][0][1]],
                        [data_2['coordinates'][1][0], data_2['coordinates'][1][1]],
                        [data_2['coordinates'][2][0], data_1['coordinates'][2][1]],
                        [data_1['coordinates'][3][0], data_1['coordinates'][3][1]]],
        'confidence': (data_1['confidence'] + data_2['confidence']) / 2
    }
    return res_data

def get_merged_data(list_of_data, image_dim=None):
    if image_dim is None:
        image_dim = [2483, 3508]            # testing purposes
    final_list = []
    for i, data_1 in enumerate(list_of_data):
        if data_1['confidence'] == -1:
            continue
        elif is_title(data_1['text']):
            final_list.append(data_1)
        else:
            for j, data_2 in enumerate(list_of_data[i + 1:]):
                if able_to_merge(data_1, data_2, image_dim):
                    data_1 = merge(data_1, data_2)
                    data_2['confidence'] = -1
            final_list.append(data_1)
    return final_list


def get_distance(point_a, point_b):
    distance = pow(((point_a[0] - point_b[0]) ** 2 + (point_a[1] - point_b[1]) ** 2), 1 / 2)
    return distance

class EvaluateResume:
    def __init__(self, _filename='resume1.png'):
        reader = OCR()
        image = cv2.imread(_filename)
        self.image_dim = [image.shape[1], image.shape[0]]
        data_list = reader.read(image)
        list_of_data = [{"wrongtext": '', 'text': '', 'coordinates': [[0, 0], [0, 0], [0, 0], [0, 0]], 'confidence': 0.0}]
        for values in data_list:
            if values[2]>=.90:
                new_data = {"wrongtext": '', 'text': values[1], 'coordinates': values[0], 'confidence': values[2]} # "correcttext": spellcheck(values[1]),
            else:
                new_data = {"wrongtext":values[1] , 'text': spellcheck(values[1]), 'coordinates': values[0], 'confidence': values[2]} # "correcttext": spellcheck(values[1]),
                print(values[1])
                print(new_data)
            list_of_data.append(new_data)
        self.all_data = get_merged_data(list_of_data, self.image_dim)
        pd.DataFrame(list_of_data).to_csv('output.csv', index=False)
    
    def get_image_dimensions(self):
        return self.image_dim

    def get_title(self, data):
        distances = []
        for current_data in self.all_data:
            if not is_title(current_data['text']):
                continue
            else:
                if data['coordinates'][1][0] > current_data['coordinates'][0][0]:
                    distance = get_distance(data['coordinates'][0], current_data['coordinates'][0])
                    base = [data['coordinates'][2][0] + 0.005 * self.image_dim[0],
                            data['coordinates'][2][1] + 0.005 * self.image_dim[1]]
                    if current_data['coordinates'][3][0] < base[0] and current_data['coordinates'][3][1] < base[1]:
                        distances.append([current_data, distance])

        final_distances = sorted(distances, key=lambda l: l[1])
        return final_distances[0][0]

    def get_key_value_pairs(self, data):
        if is_title(data['text']):
            return None
        else:
            val = self.get_title(data)
            res = {get_column(val['text']): data['text']}
            return res

def actual_main():
    all_pairs = []
    evaluator = EvaluateResume(_filename='resume1.png')
    for data in evaluator.all_data:
        res = evaluator.get_key_value_pairs(data)
        all_pairs.append(res)
        pd.DataFrame(all_pairs).to_csv('outputpairs.csv', index=False)
        print(res)

if __name__ == "__main__":
    actual_main()

100%|██████████| 27/27 [00:00<00:00, 8898.81it/s]


雁歴書
{'wrongtext': '雁歴書', 'text': '履歴書', 'coordinates': [[215, 270], [408, 270], [408, 348], [215, 348]], 'confidence': 0.10516757216668074}


100%|██████████| 27/27 [00:00<00:00, 2212.88it/s]


ふりがな
{'wrongtext': 'ふりがな', 'text': 'ふりがな', 'coordinates': [[239, 382], [423, 382], [423, 437], [239, 437]], 'confidence': 0.448096364736557}


100%|██████████| 27/27 [00:00<00:00, 5879.56it/s]


...

株式会社パラダイスプラン (沖縄県宮古島市) 入社
{'wrongtext': '株式会社パラダイスプラン (沖縄県宮古島市) 入社', 'text': '株式会社パラダイスプラン (沖縄県宮古島市) 入社', 'coordinates': [[608, 2670], [1627, 2670], [1627, 2739], [608, 2739]], 'confidence': 0.6709011427846866}


100%|██████████| 27/27 [00:00<00:00, 18546.71it/s]


店_運営
{'wrongtext': '店_運営', 'text': '店_運営', 'coordinates': [[608, 2886], [813, 2886], [813, 2948], [608, 2948]], 'confidence': 0.1612876057624817}


100%|██████████| 27/27 [00:00<00:00, 269.78it/s]


マーケティング 都訳 通訳などを担当
{'wrongtext': 'マーケティング 都訳 通訳などを担当', 'text': 'マーケティング 都訳 通訳などを担当', 'coordinates': [[830, 2885], [1625, 2885], [1625, 2951], [830, 2951]], 'confidence': 0.45542341769175254}
None
{'untitled': '履歴書'}
{'untitled': '令和 3年8月31日'}
{'untitled': '現在'}
None
{'furigana': 'テスト'}
{'furigana': '氏名'}
{'furigana': 'てすと'}
None
{'birthday': '6年 4月 23日生 (満27歳)'}
{'birthday': '性別 男'}
None
{'furigana': 'ヒョウゴケンコウベシニシクオオツワ'}
{'furigana': 'TEL・'}
None
{'address': 'テ 651-2112'}
None
{'address': '兵庫県神戸市西区大津和1丁目'}
None
None
{None: 'TEL:'}
None
{'address': 'テ'}
{'furigana': '現住所以外に連絡を必要とする場合のみ記入)'}
{'address': '同上'}
{None: '方 FAX'}
None
{'year': '月'}
{'year': '学歴 ・ 職歴 (各別にまとめて書く)'}
None
{'year': '2012'}
{'year': '9'}
{'year': '香港寝曾大鼻國際鼻院 環境保育学科 入学'}
{'year': '2014'}
{'year': '香港8曾大鼻國際鼻院 環境保育学科 卒業 (理進学士 Distinction)'}
{'year': '2015'}
{'year': '9'}
{'year': '香港中文大鼻文学部 哲学学科 入学'}
{'year': '2017'}
{'year': '4'}
{'year': '亜細亜大学 留学生別科 交換留学で来日'}
{'year': '2018'}
{'year': '3'}
{'year': '亜細亜大学 留学生別科 修了'

In [50]:
dfnew= pd.read_csv('outputpairs.csv')
dfnew.columns=['inp']
dfnew['inp']= dfnew['inp'].str.replace('{', '')
dfnew['inp']= dfnew['inp'].str.replace('}', '')
dfnew['inp']= dfnew['inp'].str.replace('\'', '')
dfnew['inp']= dfnew['inp'].fillna('')
dfnew['key']= dfnew['inp'].str.split(':').str[0]
dfnew['value']= dfnew['inp'].str.split(':').str[1]
dfnew

,inp,key,value
0,,,NaN
1,,,NaN
2,,,NaN
3,,,NaN
4,,,NaN
...,...,...,...
58,,,NaN
59,year: 2020,year,2020
60,,,NaN
61,,,NaN


In [51]:
dfnew.dropna(inplace=True)
dfnew.reset_index(inplace=True, drop=True)
dfnew

,inp,key,value
0,birthday: 6年 4月 23日生 (満27歳),birthday,6年 4月 23日生 (満27歳)
1,address: テ 651-2112,address,テ 651-2112
2,address: 兵庫県神戸市西区大津和1丁目,address,兵庫県神戸市西区大津和1丁目
3,year: 2012,year,2012
4,year: 2014,year,2014
5,year: 2015,year,2015
6,year: 2017,year,2017
7,year: 2018,year,2018
8,year: 2018,year,2018
9,year: 2020,year,2020


In [39]:
df= pd.read_csv('outputpairs.csv')
df.columns=['inp']
df['inp']= df['inp'].str.replace('{', '')
df['inp']= df['inp'].str.replace('}', '')
df['inp']= df['inp'].str.replace('\'', '')
df['inp']= df['inp'].str.replace('\'', '')
df['inp']= df['inp'].fillna('')
df['key']= df['inp'].str.split(':').str[0]
df['value']= df['inp'].str.split(':').str[1]
df

,inp,key,value
0,,,NaN
1,: 履歴書,,履歴書
2,: 令和 3年8月31日,,令和 3年8月31日
3,: 現在,,現在
4,,,NaN
...,...,...,...
58,職歴: 店_運営 マーケティング 都訳 通訳などを担当,職歴,店_運営 マーケティング 都訳 通訳などを担当
59,年: 2020,年,2020
60,年: 5,年,5
61,年: 会社都合により退社,年,会社都合により退社


In [40]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df

,inp,key,value
0,: 履歴書,,履歴書
1,: 令和 3年8月31日,,令和 3年8月31日
2,: 現在,,現在
3,ふりがな: テスト,ふりがな,テスト
4,ふりがな: 氏名,ふりがな,氏名
5,ふりがな: てすと,ふりがな,てすと
6,平成: 6年 4月 23日生 (満27歳),平成,6年 4月 23日生 (満27歳)
7,平成: 性別 男,平成,性別 男
8,りがな: ヒョウゴケンコウベシニシクオオツワ,りがな,ヒョウゴケンコウベシニシクオオツワ
9,りがな: TEL・,りがな,TEL・
